In [1]:
%pip install numpy matplotlib scapy[basic] tqdm

  Using cached https://files.pythonhosted.org/packages/62/20/4d43e141b5bc426ba38274933ef8e76e85c7adea2c321ecf9ebf7421cedf/numpy-1.18.1-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/7e/07/4b361d6d0f4e08942575f83a11d33f36897e1aae4279046606dd1808778a/matplotlib-3.1.3-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/cd/80/5bb262050dd2f30f8819626b7c92339708fe2ed7bd5554c8193b4487b367/tqdm-4.42.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/5d/bc/1e58593167fade7b544bfe9502a26dc860940a79ab306e651e7f13be68c2/pyparsing-2.4.6-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/f7/d2/e07d3ebb2bd7af696440ce7e754c59dd546ffe1bbe732c8ab68b9c834e61/cycler-0.10.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d4/70/d60450c3dd48ef87586924207ae8907090de0b306af2bce5d134d78615cb/python_dateutil-2.8.1-py2.py3-none-any.whl
  Using cached https://f

In [12]:
from tqdm import tqdm
from scapy.all import PcapReader


def accumulate(file, *accumulators):
    with PcapReader(file) as reader, tqdm(unit='packets', desc=file) as pbar:
        for packet in reader:
            for accumulator in accumulators:
                accumulator(packet)
            pbar.update()

In [18]:
from scapy.layers.dot11 import Dot11ProbeReq
from collections import Counter


class Accumulator:
    def __init__(self):
        self.timestamps = []
        self.mac_addrs = []
    
    def __call__(self, packet):
        if packet.haslayer(Dot11ProbeReq):
            self.timestamps.append(packet.time)
            self.mac_addrs.append(packet.addr2)

In [19]:
commute_16122019 = Accumulator()
commute_17122019 = Accumulator()
commute_18122019 = Accumulator()

accumulate('capture-0-a4.pcap', commute_16122019)
accumulate('capture-1-a4.pcap', commute_17122019)
accumulate('capture-2-a4.pcap', commute_18122019)

capture-0-a4.pcap: 57279packets [00:58, 972.70packets/s] 
capture-1-a4.pcap: 67268packets [01:10, 956.89packets/s] 
capture-2-a4.pcap: 59223packets [01:02, 945.79packets/s] 


In [20]:
def duration(timestamps):
    return max(timestamps) - min(timestamps)


def unique_mac_addrs(mac_addrs):
    return Counter(mac_addrs)


print(duration(commute_16122019.timestamps), len(unique_mac_addrs(commute_16122019.mac_addrs)))
print(duration(commute_17122019.timestamps), len(unique_mac_addrs(commute_17122019.mac_addrs)))
print(duration(commute_18122019.timestamps), len(unique_mac_addrs(commute_18122019.mac_addrs)))

5516.539718 14825
5496.147869 17308
4705.994265 15888
